## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ushuaia,AR,-54.8000,-68.3000,35.26,87,90,6.91,overcast clouds
1,1,Albany,US,42.6001,-73.9662,79.75,67,5,1.95,clear sky
2,2,Margate,GB,51.3813,1.3862,71.69,74,100,3.00,overcast clouds
3,3,Ponta Delgada,PT,37.7333,-25.6667,75.92,78,75,12.66,broken clouds
4,4,Bluff,NZ,-46.6000,168.3333,47.80,81,95,16.26,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Albany,US,42.6001,-73.9662,79.75,67,5,1.95,clear sky
3,3,Ponta Delgada,PT,37.7333,-25.6667,75.92,78,75,12.66,broken clouds
6,6,Luang Prabang,LA,19.8856,102.1347,75.24,99,75,1.81,broken clouds
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,85.96,47,0,11.50,clear sky
16,16,Bethel,US,41.3712,-73.4140,85.10,68,1,1.99,clear sky
17,17,East London,ZA,-33.0153,27.9116,83.34,14,0,13.80,clear sky
27,27,Kavaratti,IN,10.5669,72.6420,81.95,81,100,19.33,light rain
29,29,Port Moresby,PG,-9.4431,147.1797,76.42,94,40,11.50,scattered clouds
35,35,Marzuq,YE,14.4000,46.4667,85.30,48,79,10.40,broken clouds
36,36,Ban Dung,TH,17.6990,103.2596,81.10,99,100,7.25,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        296
City           296
Country        295
Lat            296
Lng            296
Max Temp       296
Humidity       296
Cloudiness     296
Wind Speed     296
Description    296
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Albany,US,42.6001,-73.9662,79.75,67,5,1.95,clear sky
3,3,Ponta Delgada,PT,37.7333,-25.6667,75.92,78,75,12.66,broken clouds
6,6,Luang Prabang,LA,19.8856,102.1347,75.24,99,75,1.81,broken clouds
10,10,Arraial Do Cabo,BR,-22.9661,-42.0278,85.96,47,0,11.50,clear sky
16,16,Bethel,US,41.3712,-73.4140,85.10,68,1,1.99,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Albany,US,79.75,clear sky,42.6001,-73.9662,
3,Ponta Delgada,PT,75.92,broken clouds,37.7333,-25.6667,
6,Luang Prabang,LA,75.24,broken clouds,19.8856,102.1347,
10,Arraial Do Cabo,BR,85.96,clear sky,-22.9661,-42.0278,
16,Bethel,US,85.10,clear sky,41.3712,-73.4140,
17,East London,ZA,83.34,clear sky,-33.0153,27.9116,
27,Kavaratti,IN,81.95,light rain,10.5669,72.6420,
29,Port Moresby,PG,76.42,scattered clouds,-9.4431,147.1797,
35,Marzuq,YE,85.30,broken clouds,14.4000,46.4667,
36,Ban Dung,TH,81.10,overcast clouds,17.6990,103.2596,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Vila Nova Hotel
Villa Oasis
Pousada Porto Praia
Courtyard by Marriott Danbury
Tu Casa
Hotels in Lakshadweep Islands
Lamana Hotel (Port Moresby)
Hotel not found... skipping.
Heuan Na Khee
Quinta da Meia Eira
Hanting Express Wenling Wanchang North Road
Apartment Av. Castelao
Heritage Park Hotel
Hospedaje Komby
Villa Enata
Hotel Voyage
Apogey Gotel' Zat
Hampton Inn & Suites Jamestown
Sea Lounge! Bar, Club & Restaurant
The Halyards Hotel
39 On Nile Guest House
Mondkor Town, Bagbo Chiefdom, Bo District
Abaco Beach Resort
Gostinitsa "Zapolyar'ye"
味処・民宿 まつや
Hotel Tesoro Los Cabos
Hotel not found... skipping.
Hotel The Royal Honours
Scoop Guest House
Hotel Chik Chik Namibe
Nabelchel Bungalows
Sheraton Boston Needham Hotel
Katsuura Hotel Mikazuki
Comfort Inn Hotel
Nuli Sapi
Seagulls Inn Namatani
Hotel not found... skipping.
Hotel not found... skipping.
La Quinta Inn by Wyndham Caldwell
JM Lodge
Motelik Grosar
Crôa Mares Hotel
Hotel Atoll Emerald Miyakojima
Siro Kena

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Albany,US,79.75,clear sky,42.6001,-73.9662,
3,Ponta Delgada,PT,75.92,broken clouds,37.7333,-25.6667,Vila Nova Hotel
6,Luang Prabang,LA,75.24,broken clouds,19.8856,102.1347,Villa Oasis
10,Arraial Do Cabo,BR,85.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
16,Bethel,US,85.10,clear sky,41.3712,-73.4140,Courtyard by Marriott Danbury
17,East London,ZA,83.34,clear sky,-33.0153,27.9116,Tu Casa
27,Kavaratti,IN,81.95,light rain,10.5669,72.6420,Hotels in Lakshadweep Islands
29,Port Moresby,PG,76.42,scattered clouds,-9.4431,147.1797,Lamana Hotel (Port Moresby)
35,Marzuq,YE,85.30,broken clouds,14.4000,46.4667,
36,Ban Dung,TH,81.10,overcast clouds,17.6990,103.2596,Heuan Na Khee


In [11]:
# 8a. Create the output File (CSV)
output_data_file = (r'C:\Users\helen\Desktop\Berkeley boot camp\Week 6_Py\World_Weather_Analysis\Vacation_Search\WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
city = clean_hotel_df[["City"]]

gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))